# top-level atrri.로 sub-module 추가

In [ ]:
import torch
from torch.nn import Module, init, Linear, Parameter, ReLU
from torch import optim

class DsANN (Module): #custom module

    def __init__(self,
                 n_in_f,  # input vector의 차원수.
                 n_out_f, # output vector의 차원수.
                 ):
        super().__init__() # required!

        self.linear0 = Linear(n_in_f, 32)
        self.relu0 = ReLU()

        self.linear1 = Linear(32, 32)
        self.relu1 = ReLU()

        self.linear2 = Linear(32, n_out_f)

        with torch.no_grad():
        #  이 블록 내에서의 연산은 자동 미분(autograd)에서 제외
            # linear0의 bias를 상수 0. 으로 초기화
            init.constant_(self.linear0.bias, 0.)

            # Xavier 초기화 방식으로 초기화
            init.xavier_uniform_(self.linear0.weight)

    def forward(self,x):
        x = self.linear0(x)
        x = self.relu0(x)
        x = self.linear1(x)
        x = self.relu1(x)
        y = self.linear2(x)
        return y

# add_module로 sub-module 추가

In [ ]:
class DsANN (Module): #custom module

    def __init__(self,
                 n_in_f,  # input vector의 차원수.
                 n_out_f, # output vector의 차원수.
                 ):
        super().__init__() # required!

        self.add_module('linear0', Linear(n_in_f, 32))
        self.add_module('relu0', ReLU())
        self.add_module('linear1', Linear(32, 32))
        self.add_module('relu1', ReLU())
        self.add_module('linear2', Linear(32, n_out_f))

        with torch.no_grad():
            # Module의 apply는 특정 함수를 자신의 submodules에 모두 적용 (재귀적).
            self.apply(self.init_weight)

    def forward(self,x):

        for c in self.children():
            x=c(x)
        return x

        # x = self.linear0(x)
        # x = self.relu0(x)
        # x = self.linear1(x)
        # x = self.relu1(x)
        # x = self.linear2(x)
        # return x

    @classmethod
    def init_weight(cls, module):
        if type(module) == torch.nn.Linear:
            init.kaiming_uniform_(module.weight, mode='fan_in', nonlinearity='relu')
            # init.ones_(module.weight)
            init.constant_(module.bias, 0)

model = DsANN(1,1)
print(model)

DsANN(
  (linear0): Linear(in_features=1, out_features=32, bias=True)
  (relu0): ReLU()
  (linear1): Linear(in_features=32, out_features=32, bias=True)
  (relu1): ReLU()
  (linear2): Linear(in_features=32, out_features=1, bias=True)
)


# nn.ModuleList, nn.ModuleDict, nn.Sequential 사용법/유의사항

# nn.Sequential

In [ ]:
import torch
import torch.nn as nn

model = nn.Sequential(
    nn.Linear(10, 20),
    nn.ReLU(),
    nn.Linear(20, 10),
)

print(model)

Sequential(
  (0): Linear(in_features=10, out_features=20, bias=True)
  (1): ReLU()
  (2): Linear(in_features=20, out_features=10, bias=True)
)


nn.Sequential은 nn.Linear와 nn.ReLU 모듈을 순차적으로 포함하고 있으며, 정의한 순서대로 순차적으로 적용

# nn.ModuleList

In [ ]:
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layers = nn.ModuleList([
            nn.Linear(10, 20),
            nn.ReLU(),
            nn.Linear(20, 10)
        ])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

model = MyModel()
print(model)

MyModel(
  (layers): ModuleList(
    (0): Linear(in_features=10, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=10, bias=True)
  )
)


일반적인 Python 리스트와 유사하게 동작하지만,

리스트에 포함된 모든 모듈이 PyTorch의 모델 구성 요소로 인식되어 올바르게 등록되고 관리됨.

이는 모델 파라미터가 자동으로 추적되며, to(), cuda(), cpu()와 같은 메서드를 사용할 수 있게 함

# nn.ModuleDict

In [ ]:
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layers = nn.ModuleDict({
            'fc1': nn.Linear(10, 20),
            'relu': nn.ReLU(),
            'fc2': nn.Linear(20, 10)
        })

    def forward(self, x):
        x = self.layers['fc1'](x)
        x = self.layers['relu'](x)
        x = self.layers['fc2'](x)
        return x

model = MyModel()
print(model)

MyModel(
  (layers): ModuleDict(
    (fc1): Linear(in_features=10, out_features=20, bias=True)
    (relu): ReLU()
    (fc2): Linear(in_features=20, out_features=10, bias=True)
  )
)


일반적인 Python 딕셔너리와 유사하게 동작하지만,

딕셔너리에 포함된 모든 모듈이 PyTorch의 모델 구성 요소로 인식되어 올바르게 등록되고 관리됨.

이는 모델 파라미터가 자동으로 추적되며, to(), cuda(), cpu()와 같은 메서드를 사용할 수 있게 함.

# 일반 list나 dict로 할 경우, 발생하는 문제

self.layers에 일반 list를 사용하면, 모델의 파라미터가 자동으로 추적되지 않음.

In [ ]:
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layers = [
            nn.Linear(10, 20),
            nn.ReLU(),
            nn.Linear(20, 10)
        ]

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

model = MyModel()
print(model)
print("Model parameters:", list(model.parameters()))  # 파라미터가 추적되지 않음

MyModel()
Model parameters: []


In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layers = [
            nn.Linear(10, 20),
            nn.ReLU(),
            nn.Linear(20, 10)
        ]

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

model = MyModel()
model_gpu = model.to('cuda')  # 일반 list를 사용하면 GPU로 이동하지 않음

In [ ]:
model_gpu

MyModel()

In [ ]:
model

MyModel()

# Module의 methods
* .parameters(recures=True)
* .name_buffers()
* .children()
* .modules()

In [ ]:
class DoubleLinear(Module):
    def __init__(self, n_in, n_out):
        super().__init__()

        tmp = [(n_in, n_out), (n_out, n_out)]
        for idx, t in enumerate(tmp):
            self.add_module(f'linear{idx}', Linear(*t)) # *t는 unpacking 문법. list, tuple 같은 iterable 객체의 요소들을 개별적으로 풀어서 함수,메소드에 전달.
            self.add_module(f'relu{idx}', ReLU())

    def forward(self,x):
        for c in self.children():
            x=c(x)
        return x

class DsANN (Module): #custom module

    def __init__(self,
                 n_in_f,  # input vector의 차원수.
                 n_out_f, # output vector의 차원수.
                 ):
        super().__init__() # required!

        self.add_module('module1', DoubleLinear(n_in_f,32))
        self.add_module('module2', Linear(32, n_out_f))

        with torch.no_grad():
            self.apply(self.init_weight)

    def forward(self,x):

        for c in self.children():
            x=c(x)
        return x

    @classmethod
    def init_weight(cls, module):
        if type(module) == torch.nn.Linear:
            init.kaiming_uniform_(module.weight, mode='fan_in', nonlinearity='relu')
            # init.ones_(module.weight)
            init.constant_(module.bias, 0)

model = DsANN(1,1)
print(model)

DsANN(
  (module1): DoubleLinear(
    (linear0): Linear(in_features=1, out_features=32, bias=True)
    (relu0): ReLU()
    (linear1): Linear(in_features=32, out_features=32, bias=True)
    (relu1): ReLU()
  )
  (module2): Linear(in_features=32, out_features=1, bias=True)
)


#  `__call__` 와 forward

In [ ]:
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(10, 1)

    def forward(self, x):
        print("forward called")
        return self.linear(x)

model = MyModel()

# 일반적인 방식 (추천 방식)
x = torch.randn(1, 10)
output = model(x)  # 내부적으로 model.__call__(x) → model.forward(x) 호출됨

forward called


# =====================================================

# Hook

In [18]:
import torch
import torch.nn as nn

torch.__version__

'2.6.0+cu124'

In [21]:
# 1. 모델 정의
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 2)

    def forward(self, x):
        return self.linear(x)

model = SimpleModel()

In [22]:
# 2. forward Pre-hook 등록
def pre_hook(module, input):
  print(f"[Pre-Hook] input: {input}")
  return input

model.linear.register_forward_pre_hook(pre_hook)

In [23]:
# 3. forward hook 등록
def forward_hook(module, input, output):
    print(f"[Forward Hook] output: {output}")

model.linear.register_forward_hook(forward_hook)

In [24]:
# 4. backward hook 등록 (module)
def bakcward_hook_module(module, grad_input, grad_output):
    print(f"[Backward Hook] grad_input: {grad_input}, grad_output: {grad_output}")

model.linear.register_backward_hook(bakcward_hook_module)

In [25]:
# 5. tensor gradient hook 등록
#    역전파 중에 해당 tensor에 도달했을 때,
#    해당 tensor의 gradient가 계산되기 위해
#    backward propagation으로 입력된 grad임.
def tensor_hook(grad):
  print(f"[Tensor Hook] grad: {grad}")
  return grad

# 이후 tensor객체에 대해 직접 register_hook 메서드로 callback을 넘김.

In [27]:
# 6. 입력 생성 및 forward/backward 실행 / Tensor Gradient Hook 등록
x = torch.tensor([[1.0, 2.0, 3.0, 4.0]], requires_grad=True)
y = model(x)
y.register_hook(tensor_hook)  # tensor에 직접 hook
loss = y.sum()
loss.backward()

[Pre-Hook] input: (tensor([[1., 2., 3., 4.]], requires_grad=True),)
[Forward Hook] output: tensor([[0.7827, 2.1173]], grad_fn=<AddmmBackward0>)
[Tensor Hook] grad: tensor([[1., 1.]])
[Backward Hook] grad_input: (tensor([1., 1.]), tensor([[0.2771, 0.2291, 0.4423, 0.4151]]), tensor([[1., 1.],
        [2., 2.],
        [3., 3.],
        [4., 4.]])), grad_output: (tensor([[1., 1.]]),)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


In [28]:
x.grad

tensor([[0.2771, 0.2291, 0.4423, 0.4151]])

In [29]:
# 7. forwaerd / backward 실행
y = model(x)
loss = y.sum()
loss.backward()

[Pre-Hook] input: (tensor([[1., 2., 3., 4.]], requires_grad=True),)
[Forward Hook] output: tensor([[0.7827, 2.1173]], grad_fn=<AddmmBackward0>)
[Backward Hook] grad_input: (tensor([1., 1.]), tensor([[0.2771, 0.2291, 0.4423, 0.4151]]), tensor([[1., 1.],
        [2., 2.],
        [3., 3.],
        [4., 4.]])), grad_output: (tensor([[1., 1.]]),)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1830: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


## 참고: Tensor의 register_hook 동작.

`register_hook`은 **"등록된 해당 텐서의 `grad_fn`으로 들어오는 역전파 grad"**에 hook이 걸림.

즉,

`y.register_hook(hook)`은

* **"역전파 중에 `y`에 도달했을 때, `y`의 `grad`가 계산되기 직전에
* 입력으로 들어오는 `grad`"**를 hook 함수에 넘김.
* 이때 넘겨지는 `grad`는 `y` 텐서의 뒤쪽 노드에서 넘겨받은 gradient임.

In [32]:
x1 = torch.tensor(2.0, requires_grad=True)
y1 = x1 ** 2
z1 = y1 * 3

def hook_fn(grad):
    print("y1에 전달된 grad:", grad)
    return grad

y1.register_hook(hook_fn)
z1.backward()

y1에 전달된 grad: tensor(3.)


위의 계산 흐름은 다음과 같음:

* `z1 = 3 * y1`, 그러므로 `dz1/dy1 = 3`
* `y1 = x1 ** 2`, 그러므로 `dy1/dx1 = 2x1 = 4`

전체적으로:

`z.backward() → y`로 전달되는 `grad_output = 3`

그걸 hook에서 "받아서" 출력함: `y`에 전달된 `grad: tensor(3.)`

# +) hook 해제

In [34]:
import torch.nn as nn

linear = nn.Linear(2, 1)

def backward_hook(mod, grad_in, grad_out):
    print("Backward hook called")

# hook 등록
handle = linear.register_full_backward_hook(backward_hook)

# 학습 과정 중...
x = torch.randn(1, 2, requires_grad=True)
y = linear(x)
y.sum().backward()  # hook 실행됨

# hook 해제
handle.remove()


Backward hook called


In [35]:
y = linear(x)
y.sum().backward() # hook 해제되어 hook 실행X

# =======================================================

## Save and Load

# state_dict로 parameters만을 저장하고 로드

In [38]:
# 필요한 library와 모듈 import
import torch
from torch import nn
from torch.nn import init
from collections import OrderedDict

# 간단한 linear regression model 정의.
# SimpleModel0와 SimpleModel1은
# 똑같은 구조이나 파라메터들의 초기값만 다름.
class SimpleModel0(nn.Module):

    def __init__(self, n_in_f, n_out_f):
        super().__init__()

        self.l0 = nn.Linear(n_in_f, n_out_f)

        const_weight = 1.
        const_bias = 0.5
        init.constant_(self.l0.weight, const_weight)
        if self.l0.bias is not None:
            init.constant_(self.l0.bias, const_bias)

    def forward(self, x):
        return self.l0(x)


class SimpleModel1(nn.Module):

    def __init__(self, n_in_f, n_out_f):
        super().__init__()

        self.l0 = nn.Linear(n_in_f, n_out_f)

        const_weight = 2.
        const_bias = 1.5

        init.constant_(self.l0.weight, const_weight)
        if self.l0.bias is not None:
            init.constant_(self.l0.bias, const_bias)

    def forward(self, x):
        return self.l0(x)


# 모델 객체를 생성하고, 이에 대한 파라메터 확인 후
# 파라메터만 저장.
model = SimpleModel0(3, 1)  # SimpleModel0 객체 생성
print("Initial model parameters:")
print(list(model.named_parameters()))
torch.save(model.state_dict(), 'model_params.pth')

# 새로운 모델 객체를 생성.
# 해당 모델 객체는 구조는 같으나, 파라메터들의 초기값은 다름.
n_model = SimpleModel1(3, 1)

print('===============')
# 이전 모델(SimpleModel0)과 새로운 모델(SimpleModel1)의 파라메터 비교
for old, new in zip(model.parameters(), n_model.parameters()):
    if not torch.equal(old, new):
        print('model and n_model w/ default init do not have parameters with the same values!')
        break
else:
    print('model and n_model w/ default init have parameters with the same values!')
print('===============') # break checker부분?

# 이전 저장한 parameters에 대한 state_dict를
# 로드하고 해당 state_dict로 새로 만든 모델의
# 파라메터를 설정하고 이전 모델과 비교.
# load parameters and restore old parameters into new model
loaded_params_ordered_dict = torch.load('model_params.pth')
print(f'{type(loaded_params_ordered_dict)=}')  # collections.OrderedDict

ret_v = n_model.load_state_dict(loaded_params_ordered_dict)
print(f'{type(ret_v)}: {ret_v}')

print('===============')
# 이전 모델(SimpleModel0)과 새로 로드된 파라메터로 설정된 모델(SimpleModel1)의 파라메터 비교
for old, new in zip(model.parameters(), n_model.parameters()):
    if not torch.equal(old, new):
        print('model and n_model do not have parameters with the same values!')
        break
else:
    print('model and n_model have parameters with the same values!')


Initial model parameters:
[('l0.weight', Parameter containing:
tensor([[1., 1., 1.]], requires_grad=True)), ('l0.bias', Parameter containing:
tensor([0.5000], requires_grad=True))]
model and n_model w/ default init do not have parameters with the same values!
type(loaded_params_ordered_dict)=<class 'collections.OrderedDict'>
<class 'torch.nn.modules.module._IncompatibleKeys'>: <All keys matched successfully>
model and n_model have parameters with the same values!


# -------------------------------------------------------

In [40]:
import torch
from torch import nn
from torch.nn import init
from collections import OrderedDict

In [41]:
class SimpleModel0(nn.Module):

  def __init__(self, n_in_f, n_out_f):

    super().__init__()

    self.l0 = nn.Linear(n_in_f, n_out_f)

    const_weight = 1.
    const_bias = 0.5

    init.constant_(self.l0.weight, const_weight)
    if self.l0.bias is not None:
      init.constant_(self.l0.bias, const_bias)

  def forward(self, x):
    return self.l0(x)

In [42]:
class SimpleModel1(nn.Module):

  def __init__(self, n_in_f, n_out_f):

    super().__init__()

    self.l0 = nn.Linear(n_in_f, n_out_f)
    self.l1 = nn.Linear(n_out_f, n_out_f)

    const_weight = 2.
    const_bias = 1.5

    init.constant_(self.l0.weight, const_weight)
    if self.l0.bias is not None:
      init.constant_(self.l0.bias, const_bias)

  def forward(self, x):
    return self.l0(x)

In [43]:
model = SimpleModel0(3, 1)

print(list(model.named_parameters()))

[('l0.weight', Parameter containing:
tensor([[1., 1., 1.]], requires_grad=True)), ('l0.bias', Parameter containing:
tensor([0.5000], requires_grad=True))]


In [44]:
ordered_dict_params = model.state_dict()
for c in ordered_dict_params.items():
  print(f'{c[0]:<10}={c[1]}')

l0.weight =tensor([[1., 1., 1.]])
l0.bias   =tensor([0.5000])


In [45]:
torch.save(model.state_dict(), 'model_params.pth')

In [46]:
n_model = SimpleModel1(3, 1)

for old, new in zip(model.parameters(), n_model.parameters()):
  if not torch.equal(old, new):
    print('model and n_model do not have parameters with the same values!')
    break
else:
  print('model and n_model have parameters with the same values!')

model and n_model do not have parameters with the same values!


In [47]:
# load parameters and restore old parameters into new model
loaded_params_ordered_dict = torch.load('model_params.pth')

incompatible_keys = n_model.load_state_dict(loaded_params_ordered_dict)
print(f'{type(incompatible_keys)}: {incompatible_keys}')
print(f'{incompatible_keys.missing_keys=}')
print(f'{incompatible_keys.unexpected_keys=}')

RuntimeError: Error(s) in loading state_dict for SimpleModel1:
	Missing key(s) in state_dict: "l1.weight", "l1.bias". 

# 수정해주신다고 해주심 (주말에 확인해서 적어두자)

# =====================================================

# model 전체를 저장하고 로드 (state_dict() 방식)

In [51]:
# 필요한 모듈 import
import torch
from torch import nn
from torch.nn import init

# 사용할 간단한 linear regression model 정의
class SimpleModel0(nn.Module):

    def __init__(self, n_in_f, n_out_f):
        super().__init__()

        self.l0 = nn.Linear(n_in_f, n_out_f)

        # 고정된 상수로 초기화
        const_weight = 2.  # 가중치 초기화 값
        const_bias = 1.5  # 편향 초기화 값

        # 초기화 적용
        init.constant_(self.l0.weight, const_weight)
        if self.l0.bias is not None:
            init.constant_(self.l0.bias, const_bias)

    def forward(self, x):
        return self.l0(x)

# 저장할 모델 생성.
model = SimpleModel0(3, 1)

# 모델의 state_dict만 저장
torch.save(model.state_dict(), 'model_state_dict.pth')

# 모델 로드 (모델 구조를 미리 정의한 후 state_dict를 로드)
n_model = SimpleModel0(3, 1)  # 모델 구조를 새로 정의한 후
n_model.load_state_dict(torch.load('model_state_dict.pth'))  # 파라미터만 로드

print(f'{type(n_model)=}, {n_model}')

# 두 모델의 parameters 비교.
for old, new in zip(model.parameters(), n_model.parameters()):
    if not torch.equal(old, new):
        print('model and n_model do not have parameters with the same values!')
        break
else:
    print('model and n_model have parameters with the same values!')

type(n_model)=<class '__main__.SimpleModel0'>, SimpleModel0(
  (l0): Linear(in_features=3, out_features=1, bias=True)
)
model and n_model have parameters with the same values!


아래는 weights_inly=False 옵션 사용

In [52]:
# 필요한 모듈 import
import torch
from torch import nn
from torch.nn import init
from collections import OrderedDict

# 사용할 간단한 linear regression model 정의
class SimpleModel0(nn.Module):

    def __init__(self, n_in_f, n_out_f):
        super().__init__()

        self.l0 = nn.Linear(n_in_f, n_out_f)

        # 고정된 상수로 초기화
        const_weight = 2.  # 가중치 초기화 값
        const_bias = 1.5  # 편향 초기화 값

        # 초기화 적용
        init.constant_(self.l0.weight, const_weight)
        if self.l0.bias is not None:
            init.constant_(self.l0.bias, const_bias)

    def forward(self, x):
        return self.l0(x)

# 저장할 모델 생성.
model = SimpleModel0(3, 1)

# 모델 저장 (모델 객체 전체를 저장)
torch.save(model, 'model.pth')

# 저장된 model 로드 (weights_only=False로 설정)
n_model = torch.load('model.pth', weights_only=False)

# 로드된 모델 타입 확인
print(f'{type(n_model)=}, {n_model}')

# 두 모델의 parameters 비교.
for old, new in zip(model.parameters(), n_model.parameters()):
    if not torch.equal(old, new):
        print('model and n_model do not have parameters with the same values!')
        break
else:
    print('model and n_model have parameters with the same values!')

type(n_model)=<class '__main__.SimpleModel0'>, SimpleModel0(
  (l0): Linear(in_features=3, out_features=1, bias=True)
)
model and n_model have parameters with the same values!


==========================================================

# state-dict 반환하는 예제 코드

In [53]:
import torch
import torch.nn as nn
from collections import OrderedDict

# 간단한 신경망 모델 클래스 정의
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        # 10차원 입력을 5차원으로 변환하는 선형 레이어 정의
        self.linear = nn.Linear(10, 5)
        # 5차원 특성에 대한 배치 정규화 레이어 정의
        self.bn = nn.BatchNorm1d(5)

    def forward(self, x):
        # 선형 변환 적용
        x = self.linear(x)
        # 배치 정규화 적용
        x = self.bn(x)
        return x

# 모델 인스턴스 생성
model = MyModel()

# 모델의 state_dict 획득
state_dict = model.state_dict()

# 파라미터와 버퍼 구분을 위한 출력 섹션

print("=== 파라미터 (학습 가능한 가중치) ===")
# named_parameters() 메서드를 통한 모든 파라미터 출력
for name, param in model.named_parameters():
    print(f"{name}: {param.shape}, requires_grad={param.requires_grad}")

print("\n=== 버퍼 (학습 불가능한 상태 값) ===")
# named_buffers() 메서드를 통한 모든 버퍼 출력
for name, buf in model.named_buffers():
    print(f"{name}: {buf.shape}, requires_grad={buf.requires_grad}")

print("\n=== 전체 state_dict 내용 ===")
# 파라미터와 버퍼 키 목록 생성
param_keys = [name for name, _ in model.named_parameters()]
buffer_keys = [name for name, _ in model.named_buffers()]

# state_dict의 각 항목에 대한 유형 구분
for key, value in state_dict.items():
    if key in param_keys:
        print(f"{key}: {value.shape} (파라미터)")
    elif key in buffer_keys:
        print(f"{key}: {value.shape} (버퍼)")
    else:
        print(f"{key}: {value.shape} (기타)")

# 예시 출력 설명:
# - linear.weight, linear.bias: 선형 레이어의 학습 가능한 파라미터
# - bn.weight, bn.bias: 배치 정규화의 학습 가능한 파라미터
# - bn.running_mean, bn.running_var: 배치 정규화의 통계적 버퍼 값
# - bn.num_batches_tracked: 배치 정규화의 추적용 버퍼 값

=== 파라미터 (학습 가능한 가중치) ===
linear.weight: torch.Size([5, 10]), requires_grad=True
linear.bias: torch.Size([5]), requires_grad=True
bn.weight: torch.Size([5]), requires_grad=True
bn.bias: torch.Size([5]), requires_grad=True

=== 버퍼 (학습 불가능한 상태 값) ===
bn.running_mean: torch.Size([5]), requires_grad=False
bn.running_var: torch.Size([5]), requires_grad=False
bn.num_batches_tracked: torch.Size([]), requires_grad=False

=== 전체 state_dict 내용 ===
linear.weight: torch.Size([5, 10]) (파라미터)
linear.bias: torch.Size([5]) (파라미터)
bn.weight: torch.Size([5]) (파라미터)
bn.bias: torch.Size([5]) (파라미터)
bn.running_mean: torch.Size([5]) (버퍼)
bn.running_var: torch.Size([5]) (버퍼)
bn.num_batches_tracked: torch.Size([]) (버퍼)
